In [5]:
import os
import pandas as pd

from tqdm import tqdm
from sqlalchemy import create_engine, schema, Table, text, tuple_

In [6]:
DATABASE_DRIVER = "ODBC Driver 17 for SQL Server"
DATABASE_SERVER_INSTANCE = "MSAPSQL145.bisad.bisinfo.org,55001"
DATABASE_NAME = "capital_iq_data"

conn_str = f"mssql+pyodbc://{DATABASE_SERVER_INSTANCE}/{DATABASE_NAME}?driver={DATABASE_DRIVER}"

In [7]:
con_mssql = create_engine(conn_str)

In [4]:
qry_naics = """
SELECT naicscd, naicsdesc, naicstat
FROM capital_iq_data.dbo.r_naiccd
"""

In [5]:
with con_mssql.connect() as con:
    result = con.execute(text(qry_naics))
    dfnaics = pd.DataFrame(result.fetchall(), columns=result.keys(), dtype='str')

In [6]:
entries = [tuple(row) for row in dfnaics[['naicscd', 'naicsdesc']].values.tolist()]

In [7]:
def insert(tree, code, description, level):
    for key in tree:
        if code.startswith(key) and code != key:
            insert(tree[key]['children'], code, description, level + 1)
            return
    tree[code] = {'description': description, 'level': level, 'children': {}}

tree = {}
for code, desc in entries:
    insert(tree, code, desc, 1)

# Flatten the tree
def flatten_tree(tree, parent=None, result=None):
    if result is None:
        result = []
    for code, info in tree.items():
        result.append({'code': code, 'description': info['description'], 'level': info['level'], 'parent': parent})
        flatten_tree(info['children'], parent=code, result=result)
    return result

flat = flatten_tree(tree)
df = pd.DataFrame(flat)

In [9]:
df.shape

(2919, 4)

In [8]:
df.head()

,code,description,level,parent
0,11,"Agriculture, Forestry, Fishing and Hunting",1,None
1,111,Crop Production,2,11
2,1111,Oilseed and Grain Farming,3,111
3,11111,Soybean Farming,4,1111
4,111110,Soybean Farming,5,11111


In [10]:
def get_hierarchy(df, code):
    """Return hierarchy as a list of (level, code, description) from first to given level."""
    hierarchy = []
    lookup = df.set_index('code')
    
    while True:
        if code not in lookup.index:
            break
        row = lookup.loc[code]
        # Handle case if multiple rows per code (unlikely here)
        if isinstance(row, pd.DataFrame):
            row = row.iloc[0]
        hierarchy.append((row['level'], code, row['description']))
        if pd.isnull(row['parent']):
            break
        code = row['parent']
    
    return list(reversed(hierarchy))  # First to last

In [11]:
qry_company = """
SELECT
    cc.companyId, 
    cc.companyName, 
    cicc.industryClassCode as NAICS,
    ccic.industryClassCodeId as capiq_indu_code, 
    cicc.industryClassCodeDesc as indu_desc
FROM
    ciqCompany cc
    JOIN ciqCompanyIndClass ccic ON cc.companyId = ccic.companyId
    JOIN ciqIndustryClassCode cicc ON ccic.industryClassCodeId = cicc.industryClassCodeId
WHERE
    cicc.industryClassTypeId = 416 -- NAICS 
    AND ccic.primaryFlag = 1
"""

In [12]:
with con_mssql.connect() as con:
    result = con.execute(text(qry_company))
    dfcomp = pd.DataFrame(result.fetchall(), columns=result.keys(), dtype='str')

In [13]:
dfcomp.shape

(4072805, 5)

In [14]:
dfcomp.head()

,companyId,companyName,NAICS,capiq_indu_code,indu_desc
0,18493,DC Venture Partners,523910,106655,Miscellaneous Intermediation
1,18495,13i Capital Corporation,523910,106655,Miscellaneous Intermediation
2,18499,21 International Holdings Inc.,513210,106533,Software Publishers
3,18501,21 Invest Sgr S.p.A.,523910,106655,Miscellaneous Intermediation
4,18507,2M Invest A/S,523910,106655,Miscellaneous Intermediation


In [15]:
dfcomp[dfcomp.companyName.str.contains("Samsung")]

,companyId,companyName,NAICS,capiq_indu_code,indu_desc
6552,91868,"Samsung Electronics Co., Ltd.",33441,105866,Semiconductor and Other Electronic Component M...
20098,426807,Samsung Venture Investment Corporation,523910,106655,Miscellaneous Intermediation
33095,874722,Samsung C&T Corporation,42,106038,Wholesale Trade
36067,878475,"Samsung SDI Co., Ltd.",33441,105866,Semiconductor and Other Electronic Component M...
36158,878587,"Samsung Securities Co.,Ltd.",523150,106643,Investment Banking and Securities Intermediation
...,...,...,...,...,...
4027697,1896684980,Samsung Electronics,449210,9621181,Electronics and Appliance Retailers
4044157,1939784562,Samsung Special Purpose Acquisition 10 Company,525990,106697,Other Financial Vehicles
4058661,1947958181,Samsung Special Purpose Acquisition 11 Company,525990,106697,Other Financial Vehicles
4070515,1956658702,Samsung Special Purpose Acquisition 12 Company...,525990,106697,Other Financial Vehicles


In [16]:
dfcomp['naics_level'] = dfcomp.apply(lambda x: get_hierarchy(df, x.NAICS), axis=1)

In [17]:
dfcomp.head()

,companyId,companyName,NAICS,capiq_indu_code,indu_desc,naics_level
0,18493,DC Venture Partners,523910,106655,Miscellaneous Intermediation,"[(1, 52, Finance and Insurance), (2, 523, Secu..."
1,18495,13i Capital Corporation,523910,106655,Miscellaneous Intermediation,"[(1, 52, Finance and Insurance), (2, 523, Secu..."
2,18499,21 International Holdings Inc.,513210,106533,Software Publishers,"[(1, 51, Information), (2, 513, Publishing Ind..."
3,18501,21 Invest Sgr S.p.A.,523910,106655,Miscellaneous Intermediation,"[(1, 52, Finance and Insurance), (2, 523, Secu..."
4,18507,2M Invest A/S,523910,106655,Miscellaneous Intermediation,"[(1, 52, Finance and Insurance), (2, 523, Secu..."


In [18]:
for i in range(5):
    dfcomp[f'naics_level_{i+1}'] = dfcomp['naics_level'].apply(
        lambda x: x[i] if len(x) > i else (None, None, None)
    )

dfcomp['level_1'], dfcomp['code_1'], dfcomp['desc_1'] = zip(*dfcomp['naics_level_1'])
dfcomp['level_2'], dfcomp['code_2'], dfcomp['desc_2'] = zip(*dfcomp['naics_level_2'])
dfcomp['level_3'], dfcomp['code_3'], dfcomp['desc_3'] = zip(*dfcomp['naics_level_3'])
dfcomp['level_4'], dfcomp['code_4'], dfcomp['desc_4'] = zip(*dfcomp['naics_level_4'])
dfcomp['level_5'], dfcomp['code_5'], dfcomp['desc_5'] = zip(*dfcomp['naics_level_5'])

# Optionally, drop the intermediate columns
dfcomp.drop([f'naics_level_{i+1}' for i in range(5)], axis=1, inplace=True)

In [19]:
dfcomp.head()

,companyId,companyName,NAICS,capiq_indu_code,indu_desc,naics_level,level_1,code_1,desc_1,level_2,...,desc_2,level_3,code_3,desc_3,level_4,code_4,desc_4,level_5,code_5,desc_5
0,18493,DC Venture Partners,523910,106655,Miscellaneous Intermediation,"[(1, 52, Finance and Insurance), (2, 523, Secu...",1,52,Finance and Insurance,2.0,...,"Securities, Commodity Contracts, and Other Fin...",3.0,5239,Other Financial Investment Activities,4.0,52391,Miscellaneous Intermediation,5.0,523910,Miscellaneous Intermediation
1,18495,13i Capital Corporation,523910,106655,Miscellaneous Intermediation,"[(1, 52, Finance and Insurance), (2, 523, Secu...",1,52,Finance and Insurance,2.0,...,"Securities, Commodity Contracts, and Other Fin...",3.0,5239,Other Financial Investment Activities,4.0,52391,Miscellaneous Intermediation,5.0,523910,Miscellaneous Intermediation
2,18499,21 International Holdings Inc.,513210,106533,Software Publishers,"[(1, 51, Information), (2, 513, Publishing Ind...",1,51,Information,2.0,...,Publishing Industries,3.0,5132,Software Publishers,4.0,51321,Software Publishers,5.0,513210,Software Publishers
3,18501,21 Invest Sgr S.p.A.,523910,106655,Miscellaneous Intermediation,"[(1, 52, Finance and Insurance), (2, 523, Secu...",1,52,Finance and Insurance,2.0,...,"Securities, Commodity Contracts, and Other Fin...",3.0,5239,Other Financial Investment Activities,4.0,52391,Miscellaneous Intermediation,5.0,523910,Miscellaneous Intermediation
4,18507,2M Invest A/S,523910,106655,Miscellaneous Intermediation,"[(1, 52, Finance and Insurance), (2, 523, Secu...",1,52,Finance and Insurance,2.0,...,"Securities, Commodity Contracts, and Other Fin...",3.0,5239,Other Financial Investment Activities,4.0,52391,Miscellaneous Intermediation,5.0,523910,Miscellaneous Intermediation


In [20]:
dfcomp.desc_1.value_counts()

desc_1
Manufacturing                                                               814219
Wholesale Trade                                                             648169
Retail Trade                                                                410474
Construction                                                                376445
Finance and Insurance                                                       251205
Professional, Scientific, and Technical Services                            249506
Administrative and Support and Waste Management and Remediation Services    176790
Information                                                                 143987
Transportation and Warehousing                                              143792
Health Care and Social Assistance                                           140451
Real Estate and Rental and Leasing                                          127361
Management of Companies and Enterprises                                     1268

In [21]:
dfcomp.desc_2.value_counts()

desc_2
Merchant Wholesalers, Durable Goods (eff from 6/15/2002)                         356743
Merchant Wholesalers, Nondurable Goods (eff from 6/15/2002)                      290373
Professional, Scientific, and Technical Services                                 249365
Administrative and Support Services                                              158141
Construction of Buildings (eff from 6/15/2002)                                   135154
                                                                                  ...  
Administration of Housing Programs, Urban Planning, and Community Development       685
Postal Service                                                                      597
Wholesale Trade Agents and Brokers (eff from 6/15/2002)                             462
Monetary Authorities-Central Bank                                                   395
Space Research and Technology                                                       145
Name: count, Length: 97, 

In [24]:
dfcomp[dfcomp.desc_2 == 'Monetary Authorities-Central Bank'].head()

,companyId,companyName,NAICS,capiq_indu_code,indu_desc,level_1,code_1,desc_1,level_2,code_2,desc_2,level_3,code_3,desc_3,level_4,code_4,desc_4,level_5,code_5,desc_5
11980,163065,Federal Reserve Bank of New York,521110,106611,Monetary Authorities-Central Bank,1,52,Finance and Insurance,2.0,521,Monetary Authorities-Central Bank,3.0,5211,Monetary Authorities-Central Bank,4.0,52111,Monetary Authorities-Central Bank,5.0,521110,Monetary Authorities-Central Bank
32314,873745,Bank of Finland,521110,106611,Monetary Authorities-Central Bank,1,52,Finance and Insurance,2.0,521,Monetary Authorities-Central Bank,3.0,5211,Monetary Authorities-Central Bank,4.0,52111,Monetary Authorities-Central Bank,5.0,521110,Monetary Authorities-Central Bank
32355,873803,Central Bank of Egypt,521110,106611,Monetary Authorities-Central Bank,1,52,Finance and Insurance,2.0,521,Monetary Authorities-Central Bank,3.0,5211,Monetary Authorities-Central Bank,4.0,52111,Monetary Authorities-Central Bank,5.0,521110,Monetary Authorities-Central Bank
32515,874019,Bank of Mauritius,521110,106611,Monetary Authorities-Central Bank,1,52,Finance and Insurance,2.0,521,Monetary Authorities-Central Bank,3.0,5211,Monetary Authorities-Central Bank,4.0,52111,Monetary Authorities-Central Bank,5.0,521110,Monetary Authorities-Central Bank
32529,874036,The Bank of England,521110,106611,Monetary Authorities-Central Bank,1,52,Finance and Insurance,2.0,521,Monetary Authorities-Central Bank,3.0,5211,Monetary Authorities-Central Bank,4.0,52111,Monetary Authorities-Central Bank,5.0,521110,Monetary Authorities-Central Bank


In [25]:
dfcomp.shape

(4072805, 20)

In [28]:
qry_compmeta = """
SELECT 
    cti.tradingItemId, 
    cti.securityId, 
    cs.companyId, 
    ce.exchangeName,
    ce.exchangeSymbol,
    tickerSymbol,
    cc.ISOCode,
    ccg.isoCountry2
FROM ciqTradingItem cti 
    join (
        SELECT cs.securityId, cs.companyId  
        FROM ciqSecurity cs 
        JOIN ciqCompany cc on cc.companyId = cs.companyId
        WHERE cs.securityName = 'Common Stock'
        -- and cc.companyStatusTypeId = 1 -- operating
        -- and cc.companyTypeId = 4 -- public company        
    ) cs on cti.securityId = cs.securityId
    join ciqExchange ce on ce.exchangeId = cti.exchangeId 
    join ciqCurrency cc on cc.currencyId = cti.currencyId 
    join ciqCountryGeo ccg on ccg.countryId  = ce.countryId 
WHERE primaryFlag  = 1 
order by ce.exchangeSymbol
"""

In [29]:
with con_mssql.connect() as con:
    result = con.execute(text(qry_compmeta))
    dfcompmeta = pd.DataFrame(result.fetchall(), columns=result.keys(), dtype='str')

In [35]:
dfcompmeta.shape

(38358, 8)

In [30]:
dfcomp2 = dfcompmeta.merge(dfcomp, on='companyId', how='inner')

In [31]:
dfcomp2.shape

(28741, 27)

In [45]:
dfcomp2[dfcomp2.companyName.str.contains("Samsung Electronics")]

,tradingItemId,securityId,companyId,exchangeName,exchangeSymbol,tickerSymbol,ISOCode,isoCountry2,companyName,NAICS,...,desc_2,level_3,code_3,desc_3,level_4,code_4,desc_4,level_5,code_5,desc_5
4195,20242977,20229558,91868,Korea Stock Exchange,KOSE,A005930,KRW,KR,"Samsung Electronics Co., Ltd.",33441,...,Computer and Electronic Product Manufacturing,3.0,3344,Semiconductor and Other Electronic Component M...,4.0,33441,Semiconductor and Other Electronic Component M...,NaN,None,None


In [51]:
dfcomp2_kr = dfcomp2[(dfcomp2.isoCountry2 == 'KR') & (dfcomp2.exchangeSymbol.isin(['KOSDAQ', 'KOSE']))].copy()

In [56]:
chunk_size = 200  # Adjust as needed
lst_dfs = []

trading_item_ids = dfcomp2_kr['tradingItemId'].unique().tolist()

def chunk_list(data, chunk_size):
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]

for chunk in tqdm(chunk_list(trading_item_ids, chunk_size), total=(len(trading_item_ids) + chunk_size - 1) // chunk_size, desc="Fetching price data"):
    placeholders = ','.join([str(x) for x in chunk])
    qry_price = f"""
    SELECT tradingItemId, pricingDate, priceOpen, priceHigh, priceLow, priceMid, priceClose, priceBid, priceAsk, volume, adjustmentFactor, VWAP
    FROM capital_iq_data.dbo.ciqPriceEquity
    WHERE tradingItemId IN ({placeholders})
    AND pricingDate > '2015-01-01'
    """
    with con_mssql.connect() as con:
        result = con.execute(text(qry_price))
        df_chunk = pd.DataFrame(result.fetchall(), columns=result.keys())
        lst_dfs.append(df_chunk)

df_price = pd.concat(lst_dfs, ignore_index=True)



Fetching price data: 100%|██████████| 14/14 [01:26<00:00,  6.17s/it]


#### Market capitalization

In [21]:
chunk_size = 200  # Adjust as needed
lst_dfs = []

company_ids = dfcomp2_kr['companyId'].unique().tolist()

def chunk_list(data, chunk_size):
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]

for chunk in tqdm(chunk_list(company_ids, chunk_size), total=(len(company_ids) + chunk_size - 1) // chunk_size, desc="Fetching market cap data"):
    placeholders = ','.join([str(x) for x in chunk])
    qry_marketcap = f"""
    SELECT companyId, pricingDate, marketCap, TEV, sharesOutstanding
    FROM capital_iq_data.dbo.ciqMarketCap
    WHERE companyId IN ({placeholders})
    AND pricingDate > '2015-01-01'
    """
    with con_mssql.connect() as con:
        result = con.execute(text(qry_marketcap))
        df_chunk = pd.DataFrame(result.fetchall(), columns=result.keys())
        lst_dfs.append(df_chunk)

df_marketcap = pd.concat(lst_dfs, ignore_index=True)

Fetching market cap data: 100%|██████████| 14/14 [00:45<00:00,  3.24s/it]


In [25]:
df_marketcap.shape

(7397243, 5)

In [58]:
df_price.shape

(4823366, 12)

In [8]:
caqiq_data_dir = r'L:\DRS\Projects\2025_drs_ai\capiq'

In [60]:
df_price.to_parquet(os.path.join(caqiq_data_dir, 'capiq_price_equity_kr.parquet'), index=False)

In [ ]:
dfcomp.to_parquet(os.path.join(caqiq_data_dir, 'comp_naics_code_all.parquet'))

In [ ]:
dfcomp2.to_parquet(os.path.join(caqiq_data_dir, 'comp_naics_code_common_stock.parquet'))

In [24]:
df_marketcap.head(2)

,companyId,pricingDate,marketCap,TEV,sharesOutstanding
0,7672534,2025-01-24,141108.609600,101429.029260,13065612
1,7672534,2025-01-25,141108.609600,101429.029260,13065612


In [26]:
df_marketcap.to_parquet(os.path.join(caqiq_data_dir, 'capiq_marketcap_kr.parquet'), index=False, engine='fastparquet')

In [ ]:
# dfcomp2 = pd.read_parquet(os.path.join(caqiq_data_dir, 'comp_naics_code_common_stock.parquet'), engine='fastparquet')
# df_price = pd.read_parquet(os.path.join(caqiq_data_dir, 'capiq_price_equity_kr.parquet'), engine='fastparquet')